In [33]:
import os

In [34]:
%pwd

'c:\\Users\\ambig\\jupiter_notebook'

In [35]:
os.chdir("../")

In [36]:
%pwd

'c:\\Users\\ambig'

In [37]:
from dataclasses import dataclass
from typing import List
from pathlib import Path
from typing import Dict

@dataclass
class DataIngestionConfig:
    root_dir:Path
    input_path : Path
    output_path: Path

In [38]:
from src.newsaggregator.constants import *
from src.newsaggregator.utils.common import read_yaml , create_directories

In [39]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config['root_dir']), 
            input_path=Path(config['input_path']),  
            output_path=Path(config['output_path']) 
        )
        return data_ingestion_config
    
    

In [40]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from src.newsaggregator import logger
from datetime import datetime

In [41]:


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.input_path = config.input_path
        self.output_path = config.output_path
        
    def extract_data(self):
        # Verify input path exists before attempting to read
        if not self.input_path.is_file():
            raise FileNotFoundError(f"Input file not found at {self.input_path}")
        
        # Load and save data
        df = pd.read_csv(self.input_path, encoding='utf-8')
        self.output_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure output directory exists
        df.to_csv(self.output_path, index=False)
        

    

In [42]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # Call the extract_data method
    data = data_ingestion.extract_data()

    # You can now use `data` for further processing
except Exception as e:
    print(f"An error occurred during data ingestion: {e}")


An error occurred during data ingestion: [Errno 2] No such file or directory: 'config\\config.yaml'
